
# 🧪 In-Browser Dashboard (JupyterLite, no Voilà)
This runs entirely in your browser. Use the controls below to filter and plot.
<style>
/* Hide code cells, keep menus/toolbars visible */
.jp-Cell-inputWrapper { display: none !important; }
</style>


In [ ]:

import json, math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as W
from IPython.display import display, Markdown
from py.utils import load_units, filter_units, FilterArgs, summarize, get_tuning_matrix

# Load data
DF, ORI = load_units("data/units.json")

# ---------- Widgets ----------
layer_w = W.SelectMultiple(
    options=["SG","G","IG"], value=("SG","G","IG"), description="Layers", rows=3
)
group_w = W.ToggleButtons(options=["All","MUL","MXH"], value="All", description="Group")
osi_w   = W.FloatRangeSlider(value=[0.2, 0.95], min=0.0, max=1.0, step=0.01, description="OSI")
hbw_w   = W.IntRangeSlider(value=[10, 100], min=0, max=180, step=5, description="HBW°")
ids_w   = W.Text(value="", placeholder="e.g., 1,2,3", description="Unit IDs")
overlay_w = W.Checkbox(value=True, description="Overlay selected")
maxn_w = W.BoundedIntText(value=100, min=1, max=1000, step=10, description="Max plot N")

# Outputs
count_out = W.HTML()
plot_out1 = W.Output()  # tuning overlays
plot_out2 = W.Output()  # OSI vs HBW
plot_out3 = W.Output()  # histogram

def parse_ids(txt):
    txt = txt.strip()
    if not txt:
        return None
    try:
        ids = [int(x.strip()) for x in txt.split(",") if x.strip()]
        return ids if ids else None
    except:
        return None

def current_selection():
    layers = list(layer_w.value) if layer_w.value else None
    group = None if group_w.value == "All" else group_w.value
    ids = parse_ids(ids_w.value)
    df = filter_units(
        DF, layers=layers, group=group, osi=tuple(osi_w.value), hbw=tuple(hbw_w.value), ids=ids
    )
    return df

def update(_=None):
    df = current_selection()
    n = len(df)
    count_out.value = f"<b>Selected:</b> {n} units"
    # restrict plotting crowd
    dfp = df.head(maxn_w.value)

    with plot_out1:
        plot_out1.clear_output(wait=True)
        if len(dfp)==0:
            display(Markdown("> No units to plot."))
        else:
            fig = plt.figure(figsize=(6,4), dpi=120)
            if overlay_w.value:
                # overlay control for all, plus mean
                A = np.stack(dfp["control"].to_list(), axis=0)
                for row in A:
                    plt.plot(ORI, row, alpha=0.15, linewidth=1)
                plt.plot(ORI, A.mean(axis=0), linewidth=2)
                plt.title(f"Control tuning (overlay up to {len(dfp)} units)")
                plt.xlabel("Orientation (deg)"); plt.ylabel("Response (a.u.)")
            else:
                # show first unit control vs laser
                u = dfp.iloc[0]
                plt.plot(ORI, u["control"], linewidth=2, label="Control")
                plt.plot(ORI, u["laser"], linewidth=2, linestyle="--", label="Laser")
                plt.title(f"Unit {u['id']}  ({u['layer']}, {u['group']})")
                plt.xlabel("Orientation (deg)"); plt.ylabel("Response (a.u.)"); plt.legend()
            plt.tight_layout()
            plt.show()

    with plot_out2:
        plot_out2.clear_output(wait=True)
        if len(dfp)==0:
            display(Markdown("> No data."))
        else:
            fig = plt.figure(figsize=(5,4), dpi=120)
            x = dfp["hbw_control"].astype(float)
            y = dfp["osi_control"].astype(float)
            plt.scatter(x, y, s=18, alpha=0.6)
            plt.xlabel("HBW (deg) — Control")
            plt.ylabel("OSI — Control")
            plt.title("OSI vs HBW (Control)")
            plt.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.show()

    with plot_out3:
        plot_out3.clear_output(wait=True)
        if len(dfp)==0:
            display(Markdown("> No data."))
        else:
            fig = plt.figure(figsize=(5,4), dpi=120)
            # histogram of peak orientations
            plt.hist(dfp["peak_ori"], bins=np.arange(-5, 365, 10), rwidth=0.9)
            plt.xlabel("Peak orientation (deg)")
            plt.ylabel("Count")
            plt.title("Peak orientation distribution")
            plt.tight_layout()
            plt.show()

# Wire callbacks
for w in (layer_w, group_w, osi_w, hbw_w, ids_w, overlay_w, maxn_w):
    w.observe(update, names="value")

ui = W.VBox([
    W.HTML("<h3>Filters</h3>"),
    W.HBox([layer_w, group_w]),
    W.HBox([osi_w, hbw_w]),
    W.HBox([ids_w, overlay_w, maxn_w]),
    count_out,
    W.HTML("<hr>"),
    W.HTML("<h3>Plots</h3>"),
    W.HBox([plot_out1, plot_out2, plot_out3]),
])

display(ui)
update()
